In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pandas as pd

# Pointwise 검사를 위한 데이터셋 준비
#prompt(Source), reference, response
data = [
    ["This account is not protected with a two-factor authentication.", 
     "2차 인증이 되지 않은 계정입니다.",
     "이중 인증으로 보호되지 않은 계정입니다."],
    ["Even the longest journey begins with a single step.",
    "천 리 길도 한 걸음부터.",
    "가장 먼 여정도 한 걸음에서 시작됩니다."],
    ["Even the longest journey begins with a single step.",
    "Лиха беда начало.",
    "Даже самая длинная дорога продолжается с одного шага."]
]
pointwise_df = pd.DataFrame(data, columns=['prompt', 'reference', 'response'])
pointwise_df

In [ ]:
from vertexai.evaluation import EvalTask, PointwiseMetric, PairwiseMetric
pointwise_ground_truth_metric_prompt = """
# Instruction
당신은 전문적인 번역 품질 평가자 입니다. 당신은 AI 모델이 번역한 내용의 품질을 평가해야 합니다.
우리는 당신에게 'Source, AI translation, Ground Truth'를 제공할 것입니다.
업무를 수행하기 위해 'Source, AI translation, Ground Truth'을 신중하게 읽고, 아래의 'Evaluation' 섹션에 정의된 'Evaluation criteria'에 근거하여 평가해야 합니다.
당신은 'Rating rubric'과 'Evaluation steps'에 기반하여 평가해야 합니다. 평가의 이유에 대해 단계별로 '한글'로 설명하고 'Rating rubric'의 순위만 선택해야 합니다.

# Evaluation
## Metric Definition
번역이 'Ground Truth'와 동일하게 번역되었는지, 그렇지 않으면 의미적으로 유사하고 간결, 명확, 캐주얼하게 작성되었는지 전반적으로 확인합니다.

## Evaluation criteria
예시 일치성: 예시로 제공되는 'Ground Truth'와 얼마나 일치하는지 확인합니다.
지시를 잘 따르는지: 'AI translation'이 'Evaluation Definition'을 잘 따라서 수행됐는지 확인합니다.

## Rating rubric
5: (매우 좋음). 'Ground Truth'와 글자수 등 표현이 정확하게 일치함
4: (좋음). 'Ground Truth'와 어순 외 차이 없음
3: (괜찮음). 속담이나 관용적 표현을 사용하지 못했지만 이해하는데 큰 문제 없음
2: (나쁨). 이해하기가 어려움
1: (매우 나쁨). 번역결과가 'Ground Truth'와 매우 다름

## Evaluation steps
STEP 1: 'Ground Truth'을 참고하여 'AI translation'이 번역한 내용을 비교합니다.
STEP 2: 평가 규칙에 따라 점수를 부여합니다.

# Source, AI translation, Ground Truth
### Source
{prompt}

### Ground Truth
{reference}

### AI translation
{response}
"""

pointwise_ground_truth_text_quality = PointwiseMetric(
    metric="pointwise_ground_truth_quality",
    metric_prompt_template=pointwise_ground_truth_metric_prompt,
)

In [ ]:
eval_task = EvalTask(dataset=pointwise_df, 
                    metrics=[pointwise_ground_truth_text_quality, "bleu", "rouge"],
                    experiment="pointwise-eval")
result_pointwise = eval_task.evaluate()

In [ ]:
result_pointwise.summary_metrics

In [ ]:
pd.options.display.max_colwidth = 300
result_pointwise.metrics_table

In [ ]:
# Pairwise 검사를 위한 데이터셋 준비
#prompt(Source), reference, response (Response B), baseline_model_response (Response A)
data = [
    ["This account is not protected with a two-factor authentication.", 
     "2차 인증이 되지 않은 계정입니다.",
     "이중 인증으로 보호되지 않은 계정입니다.",
     "이 계정은 이중 인증으로 안전하게 보호되지 않습니다."],
    ["Even the longest journey begins with a single step.",
    "천 리 길도 한 걸음부터.",
    "가장 먼 여정도 한 걸음에서 시작됩니다.",
    "가장 긴 여행조차도 한 걸음부터 시작됩니다."],
    ["Even the longest journey begins with a single step.",
    "Лиха беда начало.",
    "Даже самая длинная дорога продолжается с одного шага.",
    "Даже самый долгий путь начинается с первого шага."]
]
pairwise_df = pd.DataFrame(data, columns=['prompt', 'reference', 'response', 'baseline_model_response'])
pairwise_df

In [ ]:
pairwise_model_compare_metric_prompt = """
# Instruction
당신은 전문적인 번역 품질 평가자 입니다. 당신은 2개의 AI 모델이 번역한 내용의 품질을 평가해야 합니다.
우리는 당신에게 'Source, Response A, Response B, Ground Truth'를 제공할 것입니다.
업무를 수행하기 위해 'Source, Response A, Response B, Ground Truth'을 신중하게 읽고, 아래의 '평가' 섹션에 정의된 '평가항목'에 근거하여 평가해야 합니다.
당신은 'Evaluation rule'과 'Evaluation steps'에 기반하여 평가해야 합니다. 평가의 이유에 대해 단계별로 '한글'로 설명하고 'Evaluation rule'의 순위만 선택해야 합니다.

# Evaluation
## Metric Definition
번역이 'Ground Truth'와 동일하게 번역되었는지, 그렇지 않으면 의미적으로 유사하고 간결, 명확, 캐주얼하게 작성되었는지 전반적으로 확인합니다.

## Evaluation criteria
예시 일치성: 예시로 제공되는 'Ground Truth'와 얼마나 일치하는지 확인합니다.
지시를 잘 따르는지: 'AI translation'이 'Evaluation Definition'을 잘 따라서 수행됐는지 확인합니다.

## Rating rubric
STEP 1: Analyze Response A based on all the Criteria.
STEP 2: Analyze Response B based on all the Criteria.
STEP 3: Compare the overall performance of Response A and Response B based on your analyses and assessment.
STEP 4: Output your preference of "A", "SAME" or "B" to the pairwise_choice field according to the Rating Rubric.
STEP 5: Output your assessment reasoning in the explanation field.

# Source, Response A, Response B, Ground Truth
### Source
{prompt}

### Ground Truth
{reference}

### Response A
{baseline_model_response}

### Response B
{response}
"""

pairwise_ground_truth_text_quality = PairwiseMetric(
    metric="pairwise_ground_truth_quality",
    metric_prompt_template=pairwise_model_compare_metric_prompt,
)

In [ ]:
eval_task = EvalTask(dataset=pairwise_df, 
                    metrics=[pairwise_ground_truth_text_quality],
                    experiment="pairwise-eval")
result_pairwise = eval_task.evaluate()

In [ ]:
result_pairwise.summary_metrics

In [ ]:
pd.options.display.max_colwidth = 300
result_pairwise.metrics_table.rename(columns={"baseline_model_response": "Response A", "response": "Response B"}).replace(['BASELINE', 'CANDIDATE'], ['A', 'B'])